## The Hamiltonian for the spring-ball oscillating system given in paper https://arxiv.org/pdf/2303.13012.pdf

Following largely from section 3 of the paper

- $ H = -\begin{bmatrix} 0 & B \\ B^\dagger & 0 \end{bmatrix} $ where dim of $B$ is $N \text{x} M$, therefore $dim(H) = (N+M) \text{x} (N+M)$
- We pad $B$ with zeros such that $dim (B) = N^2 \text{x}N^2$ (according to appendix A1 and A4), therefore $dim(H) = 2N^2 \text{x} 2N^2$
- Now for $B$ we have, $BB^\dagger = A$, $ \begin{equation}
  \sqrt{M}B|j,k\rangle =
    \begin{cases}
      \sqrt{k_{jj}}|j\rangle & \text{if } j=k \\
      \sqrt{k_{jk}}(|j\rangle-|k\rangle) & \text{if } j<k\\
    \end{cases}       
\end{equation} $ and elements of $B^\dagger$ are either $\sqrt{k_{jk}/m_j}$ or $0$.

- $ A$ = $\sqrt{M}^{-1] F \sqrt{M}^{-1} $.

- $M$ is diagonal matrix of masses $(m_{jj}>0)$ and $F$ is the $N \text{x} N$ matrix whose diagonal and off-diagonal entries are $f_{jj} = \sum_k \kappa_{jk}$ and $f_{jk} = -\kappa_{jk}$,respectively. 

for the second point in $|j,k\rangle$ state is in vector [00,11,22,33,01,02,03,12,13,23]

#### Case : $$d = 1, E = 1, n = 2 \implies N = 4, \text{ no. of springs = 4 }, m_i = 1 \forall i \in [1,N], k_{i,j} \in [1,4], j>i, \forall (i,j) $$
$$\dot{\vec{x}}(0) = \begin{bmatrix} 1 \\ -1 \\ 0 \\  0 \end{bmatrix} \text{ and } \vec{x} = \begin{bmatrix} 0 \\ 0 \\ 0 \\ 0 \end{bmatrix}$$ 

In [141]:
import numpy as np
import scipy as sc
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import HGate
from qiskit import QuantumCircuit

In [144]:
class Hamiltonian_Formulation:
    def __init__(self, n, K_matrix, M):
        self.N = 2**n
        self.K_matrix = K_matrix
        self.M = M
        
    def compute_F_matrix(self):
        N = self.N
        K = self.K_matrix
        # Matrix F
        F = np.zeros((N,N))
        for j in range(N):
            for k in range(N):
                if j==k:
                    F[j][k] = sum(K[j]) 
                else:
                    F[j][k] = -K[j][k]
        self.F_matrix = F
    
    def compute_B_matrix(self):
        N = self.N
        K = self.K_matrix
        sqM_B = np.zeros((N,int(N*(N+1)/2)))
        count = 0 #this will keep track of the coloumn of B based on |j,k>
        for j in range(N):
            for k in range(N):
                if j==k:
                    sqM_B[j][k] = np.sqrt(K[j][j])
                elif j<k:
                    sqM_B[j][N+count] = np.sqrt(K[j][k])
                    sqM_B[k][N+count] = -np.sqrt(K[j][k])
                    count+=1
        sqM_inv = np.linalg.inv(np.sqrt(M))
        #B matrix
        B = np.matmul(sqM_inv,sqM_B)
        #B dagger matrix
        B_dag = np.matrix(B).getH()
        self.B_matrix = B
        self.B_dag_matrix = B_dag
        
    def compute_Hamiltonian(self):
        #calling compute_B_matrix
        self.compute_B_matrix()
        ## doing the padding
        B = self.B_matrix
        B_dag = self.B_dag_matrix
        
        pad_B = np.pad(B,((0,int(N**2-N)),(0,int(N**2-(N*(N+1)/2)))),"constant")
        pad_B_dag = np.matrix(pad_B).getH()
        temp1 = np.concatenate((0*np.identity(N**2),-pad_B),axis = 1)
        temp2 = np.concatenate((-pad_B_dag, 0*np.identity(N**2)),axis = 1)
        # getting the hamiltonian
        H = np.concatenate((temp1,temp2),axis = 0)
        self.H_matrix = H

In [154]:
n = 2
N = 2**n
m = 1
E = 1
#mass matrix
M = np.diag(np.full(N,m)) 
#K matrix (spring constants)
k11 = 2
k12 = 1
k23 = 3
k34 = 2
K = np.array(([k11,k12,0,0],[k12,0,k23,0],[0,k23,0,k34],[0,0,k34,0]))


ham_formulation = Hamiltonian_Formulation(n, K, M)
ham_formulation.compute_Hamiltonian()
H_matrix = ham_formulation.H_matrix
H_matrix[1]
print(len(H_matrix))

32


## State Preparation

In [156]:
initial_state = np.array([1/np.sqrt(2),-1/np.sqrt(2)] + [0 for i in range(2*N**2-2)])
print(len(initial_state))
# use https://github.com/Qiskit/qiskit/issues/11735#issuecomment-1992145075 or Classiq
initial_state

qc = QuantumCircuit(2*n+1)
qc.h(0)
qc.z(0)

init_state = Statevector.from_instruction(qc)

32


## Conditions for testing
- t = 5
- for n = 2, 5, 10
- accuracy = 99%
- note down accuracy, depth of circuit, number of gates (single and 2 qubit)

In [161]:
t=5

# exact_times = np.linspace(0, t, 101)
# We define a slightly denser time mesh
exact_times = np.linspace(0, t, 101)

# We compute the exact evolution using the exp
final_state = init_state.evolve(sc.linalg.expm(-1j * t * H_matrix)) 

In [162]:
final_state.is_valid()

True